## Efficient Yelp API Calls (Core)
For this assignment, you will be working with the Yelp API.

As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

### Load Credentials and Create Yelp API Object

In [2]:
# Load API Credentials
with open(r"C:\Users\bandi\.secret\yelp_api.json") as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)
yelp_api

### Define Search and File Paths

In [18]:
# set our API call parameters 
LOCATION = 'Greenville, SC'
TERM = 'Sushi'

In [19]:
# Name the file to save results
JSON_FILE = "Data/results_SC_sushi.json"
JSON_FILE

'Data/results_SC_sushi.json'

### Check if Json File exists and Create it if it doesn't

In [20]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)

## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f) 
        
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_SC_sushi.json not found. Saving empty list to file.


### Load JSON FIle and account for previous results

In [21]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


### Figure out how many pages of results we will need

In [22]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

HTTPError: 429 Client Error: Too Many Requests for url: https://api.yelp.com/v3/businesses/search?location=Greenville%2C+SC&term=Sushi&offset=0

In [9]:
## How many results total?
total_results = results['total']
total_results

14700

In [10]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [15]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((total_results - n_results) / results_per_page)
n_pages

735

### Add this page of results to .json file

In [16]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f,indent = 4)

### For Loop to call each page

In [17]:
for i in tqdm_notebook(range(1, n_pages + 1)):

    ## The block of code we want to TRY to run
    try:        
        # Introduce a short delay to respect API rate limits
        time.sleep(0.2)
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as file:
            previous_results = json.load(file)
        
        ## Save number of results to use as offset
        n_results = len(previous_results)
        
        ## Use n_results as the OFFSET 
        results = yelp_api.search_query(location = LOCATION,
                                        term = TERM,
                                        offset = len(previous_results))

        ## Append new results and save to file
        previous_results.extend(results['businesses'])
        with open(JSON_FILE, 'w') as file:
            json.dump(previous_results, file)

            
## What to do if we get an error/exception.
    except Exception as e:
        # check if we are at rate limit
        if 'Too Many Requests for url' in str(e):
            print('Rate limit exceeded. Stop data collection.')
            break
        else:
            print(f'an error occured {e}')
            # optionally handle error differently
            continue

  0%|          | 0/735 [00:00<?, ?it/s]

an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&te

an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&te

an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&te

an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&te

an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&te

an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&te

an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&te

an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&term=Burger&offset=1000
an error occured 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=New+York%2C+NY&te

In [ ]:
results = yelp_api.search_query(location='New York, NY',
                                       term='Burger')
print(type(results))
results.keys()

In [ ]:
pizza = pd.DataFrame(results['businesses'])
pizza.head(2) 

In [ ]:
## total number of matching businesses
results['total']

In [ ]:
## how many businesses in our results
len(results['businesses'])

In [ ]:
# add offset to our original api call
results = yelp_api.search_query(location='New York, NY',
                                       term='Pizza',
                                       offset = 20)

In [ ]:
pizza20 = pd.DataFrame(results['businesses'])
pizza20.head(2)

In [ ]:
## concatenate the previous results and new results. 
results_pizza = pd.concat([pizza, pizza20],
                      ignore_index=True)
display(results_pizza.head(3), results_pizza.tail(3))

In [ ]:
# Specify directory and filename
directory = 'Data'

# Ensure that the 'Data' directory exists
os.makedirs(directory, exist_ok = True)

In [ ]:
# save the results to records-style json file
results_pizza.to_json("Data/yelp-search-new-york-pizza.json", orient='records')